## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root artifacts`

In [1]:
import mlflow


# mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/756373249208193818', creation_time=1684321280058, experiment_id='756373249208193818', last_update_time=1684321280058, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1684321190346, experiment_id='0', last_update_time=1684321190346, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 'file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/756373249208193818/6b7884e8a8aa49f6810d0c9bed2e6eef/artifacts'


/home/fn/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/756373249208193818', creation_time=1684321280058, experiment_id='756373249208193818', last_update_time=1684321280058, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1684321190346, experiment_id='0', last_update_time=1684321190346, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5001")

In [13]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1684322844803, description='', last_updated_timestamp=1684322844948, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1684322844948, current_stage='None', description='', last_updated_timestamp=1684322844948, name='v1', run_id='6b7884e8a8aa49f6810d0c9bed2e6eef', run_link='', source='file:///home/fn/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/756373249208193818/6b7884e8a8aa49f6810d0c9bed2e6eef/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='v1', tags={}>]

In [16]:
run_id = client.search_runs(experiment_ids='1')[0].run_id
# mlflow.register_model(
#     model_uri=f"runs:/{run_id}/models",
#     name='iris-classifier'
# )

IndexError: list index out of range